# Predicting Forest Coverage Type to Predict Ecological Shifts

by: Carter Koehler

## Data Description and Prediction Task

This dataset contains information on the forest coverage types from the Roosevelt National Forest in Colorado. The data consists of $12$ explanatory variables related to cartographic measurements of a particular area and a categorical encoding of what type of tree cover that area has. In particular, it has features on:

- elevation
- grade of slope (slope)
- direction of slope (aspect)
- distance to water
- distance to roads
- distance to fire points
- shade coverage at various times in the day
- wilderness area
- soil type

All of the categorical data (with the exception of the target variable) has already been one-hot encoded, meaning the the data we are dealing with already has quite a few more than $12$ columns.

All of this is well and good, but it begs the question why we would want to be able to predict the forest coverage of a particular block of forest. One possible application is as follows: If it happens that we are able to predict how a particular area of forest is covered, this gives us a good qualitative description of what flora is present in that area. However, if the cartographic features of that area were to change due to human or other influence, such that we would predict a different coverage type than is there currently, we should predict that that area is about to go through a severe ecological change. This will give administrators and rangers a chance to prepare for major changes in the forest's wildlife. Unfortunately, this application is difficult to test for directly, so we will settle for predicting the classes as is. Additionally, since we will be using this to test for changes in the environment, we will care as much about false negatives as false positives, since the precise classification will inform whether or nota change is correctly detected.

This of course brings into question what we might use as a suitable baseline. We will see in the exploratory stages of the analysis that the distribution of cover type changes significantly between wilderness areas. We will use this to derive our baseline, which will predict the most frequent cover type for each wilderness area. For our algorithm to be considered as successfully fulfilling our task, it should be able to beat this baseline to within a statistically significant margin.

Much of the analysis in this notebook, including the above description, follows from [work completed for the previous project](https://github.com/carterkoehler/evaluation-and-mlp) on using Multi-Layer Perceptrons. More extensive visualizations of the feature distributions and other aspects of the dataset can be found there. Only ones that are directly applicable to the current analysis will be reproduced here.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rcParams
rcParams['figure.figsize'] = 10, 8
rcParams['axes.grid'] = True

import os
import sys
from pprint import pprint

from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.layers import Embedding, Flatten, Merge, concatenate
from keras.models import Model

Using TensorFlow backend.


In [3]:
filename = os.path.join(os.pardir, 'data', 'covtype.csv')
df_cov = pd.read_csv(filename)

## Picking Evaluation Methods